In [9]:
!pip freeze | grep scikit-learn

scikit-learn==1.1.1


In [10]:
import pickle
import pandas as pd

In [11]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [12]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [13]:
df = read_data('data/fhv_tripdata_2021-02.parquet')

In [14]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

### Q1 Notebook

In [17]:
import numpy as np
import pandas as pd
print("Mean predicted duration for this dataset: {}".format(np.mean(y_pred)))

Mean predicted duration for this dataset: 16.191691679979066


### Q2 Preparing the output

In [18]:
year = 2021
month = 2
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
output_file = f'output/{year:04d}-{month:02d}.parquet'

In [19]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [20]:
import os
# get file stats
stats = os.stat(output_file)
print('Size of file is', stats.st_size, 'bytes')

Size of file is 19711507 bytes


### Q3 Creating the scoring script
python score.py 2021 2

### Q4 Virtual environment
sha256:0791fbd1e43bf74b3502133207e378901272f3c156c4df4954cad833b1380207

### Q5 Parametrize the script

In [21]:
! python score.py 2021 03

/home/mlops_zoomcamp/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/mlops_zoomcamp/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
done applying model, mean prediction: 16.298821614015107


## Q6 Docker contaner

In [22]:
!docker run 04_test 2021 04

/usr/local/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
done applying model, mean prediction: 9.967573179784523
